# Lab 5: Generative Objects

Object-oriented programming (OOP) is a powerful paradigm for organizing code: you group related data and the methods that operate on that data into classes. In the world of LLMs, a similar organizational principle emerges—especially when you want to combine structured data with LLM-powered "tools" or operations. This is where Mellea's MObject abstraction comes in.

The MObject Pattern: You should store data alongside its relevant operations (tools). This allows LLMs to interact with both the data and methods in a unified, structured manner. It also simplifies the process of exposing only the specific fields and methods you want the LLM to access.

The MObject pattern also provides a way of evolving existing classical codebases into generative programs. Mellea's @mify decorator lets you turn any class into an MObject. If needed, you can specify which fields and methods are included, and provide a template for how the object should be represented to the LLM.

In [ ]:
# Import necessary libraries
import pandas as pd
import mellea
from mellea.stdlib.mify import mify
from io import StringIO

# Display utilities
from IPython.display import display, Markdown

# Format code cells with black
import jupyter_black

jupyter_black.load()

## Extract table from the document

We can use docling to extract documents from the table.

In [ ]:
@mify(fields_include={"table"}, template="{{ table }}")
class MyCompanyDatabase:
    """
    Simple in-memory company sales table wrapped as an MObject.

    The class stores a CSV table (as a string) and exposes utility methods to
    parse and update that table. The @mify decorator controls which fields are
    visible to the LLM and how the object is rendered.

    Attributes:
        table (str): CSV formatted table with columns "Store" and "Sales".
    """

    table: str = """Store,Sales
Northeast,250
West,300
Southeast,80
Midwest,420
Southwest,150
Pacific,500"""

    def __init__(self, *, table: str | None = None):
        # Allow overriding the default CSV table when constructing the object.
        if table is not None:
            self.table = table

    def _parse_table(self, table: str) -> pd.DataFrame:
        """
        Parse the CSV table string into a pandas DataFrame.

        Args:
            table: CSV-formatted string representing the table.

        Returns:
            pd.DataFrame: Parsed table with columns such as "Store" and "Sales".
        """
        # Read CSV string into a DataFrame.
        df = pd.read_csv(StringIO(table), sep=",")
        return df

    def update_sales(self, store: str, amount: int) -> "MyCompanyDatabase":
        """
        Update the sales for a specific store and return a new database instance.

        This method finds the row matching `store`, updates its "Sales" value to
        `amount`, and returns a new MyCompanyDatabase containing the updated CSV.
        Returning a new instance keeps the operation effectively immutable.

        Args:
            store: Name of the store to update (matches the "Store" column).
            amount: New sales value to set (kept as string to preserve CSV format).

        Returns:
            MyCompanyDatabase: A new instance with the updated table CSV.
        """
        # Parse current table, update the Sales cell, and return a new instance.
        df = self._parse_table(self.table)
        df.loc[df["Store"] == store, "Sales"] = int(amount)
        return MyCompanyDatabase(table=df.to_csv(sep=",", index=False, header=True))

In [ ]:
# Create a Mellea model using the granite3.3:8b model and the ollama inference engine
m = mellea.start_session()

# Example usage
db = MyCompanyDatabase()
print(m.query(db, "What were sales for the Northeast branch this month?"))

# Update the sales for the Northeast branch
db = m.transform(db, "Update the northeast sales to 1250.")
print(m.query(db, "What were sales for the Northeast branch this month?"))

# Exercises

1. TBD